# Библиотеки

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from keras.models import Model, Input
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization

# Загрузка данных